## Import necessary libraries

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# pandas
import pandas as pd

# python imports
import re
import json
from collections import Counter
from matplotlib import pyplot as plt

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

import matplotlib.animation as animation
from matplotlib.widgets import Slider

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import matplotlib.patches as patches
from matplotlib import rc
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

rc('animation', html='jshtml')
stopwords = set(STOPWORDS)


import pickle

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


### Installing NLTK Packages

In [4]:
#import nltk
#from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

## Reading in the corpus

For our example,we will be using the Wikipedia page for chatbots as our corpus. Copy the contents from the page and place it in a text file named ‘chatbot.txt’. However, you can use any corpus of your choice.

In [5]:
f=open('chatbot.txt','r',errors = 'ignore')
#f = pd.read_fwf('..input/chatbot.txt',delimiter = "\t")
raw=f.read()
raw = raw.lower()# converts to lowercase
raw

'a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods. such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test. chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition. some chatterbots use sophisticated natural language processing systems, but many simpler systems scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database.\n\nthe term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversational programs.today, most chatbots are either accessed via virtual assistants such as google assistant and amazon

In [6]:
raw ="""How dangerous is the coronavirus Covid-19 is deadly, although fatality rates skyrocket for the elderly and those with compromised immune systems.How has the coronavirus affected you Well if you ask me personally, the outbreak of novel Coronavirus, has affected me significantly. Both in respect of career and in respect of mental health. Is there a treatment for the Coronavirus They are experimenting with various medicines and it seems some anti-malarial drugs show promise. However, they help only a little bit, as evidenced by the mortality rate in places like Italy or Spain. They are best if administered early, before the lungs are damaged. Is there a vaccine for the coronavirus We don’t “find” vaccines. That’s the language yellow media uses, but it’s far from the truth.
We know the virus, and we know what it looks like. Now we need to produce a sufficient amount of attenuated virii. For that, we inject the virus into a foreign host, like a chicken embryo in an egg. Most viruses will die, because the host is so different from humans, that it can not work. Some will mutate to work, and those multiply.How did coronaviruses get their name The coronavirus family was discovered first in 1960s but we don’t know where they come from. They get the name given their crown-like (“corona”) shape. Following a December 2019 outbreak in China, the World Health Organization identified a new type called 2019 novel coronavirus (2019-nCoV).What are coronaviruses Coronavirus is a name for a group of viruses that target mammals, including cats, dogs, and humans. They’re well known for causing potentially fatal conditions like kidney failure, pneumonia but also mundane things like some instances of the common cold.
Does coronavirus cause death Statistically speaking, you have a 98% chance of survival, and 80% chance that you won’t need medical care at all even if you get infected, so stay positive, stay at home as much as possible, eat healthy, treat everyone as if they are infected, treat other as if you are infected. Many people get infected and get better without even knowing they had it.What can one do to prepare for the coronavirus? What should one buy or do regularly So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.What should I buy to prepare for a coronavirus epidemic if I had to stay in my home for a very long time So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.
What are the symptoms of Coronavirus The symptoms of this virus include high fever, respiratory problems, shortness of breath and coughing. In extreme cases, it can even lead to pneumonia, kidney failure and death.To prevent infection from spreading, one should wash hands regularly, cover mouth and nose while coughing and sneezing and eat properly cook meat and eggs."""

## Tokenisation

In [7]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Preprocessing

We shall now define a function called LemTokens which will take as input the tokens and return normalized tokens.

In [8]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## Keyword matching

Next, we shall define a function for a greeting by the bot i.e if a user’s input is a greeting, the bot shall return a greeting response.ELIZA uses a simple keyword matching for greetings. We will utilize the same concept here.

In [9]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "hi Babe", "how are you?")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me","Thanks for asking"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## Generating Response

### Bag of Words
After the initial preprocessing phase, we need to transform text into a meaningful vector (or array) of numbers. The bag-of-words is a representation of text that describes the occurrence of words within a document. It involves two things:

* A vocabulary of known words.

* A measure of the presence of known words.

Why is it is called a “bag” of words? That is because any information about the order or structure of words in the document is discarded and the model is only **concerned with whether the known words occur in the document, not where they occur in the document.**

The intuition behind the Bag of Words is that documents are similar if they have similar content. Also, we can learn something about the meaning of the document from its content alone.

For example, if our dictionary contains the words {Learning, is, the, not, great}, and we want to vectorize the text “Learning is great”, we would have the following vector: (1, 1, 0, 0, 1).


### TF-IDF Approach
A problem with the Bag of Words approach is that highly frequent words start to dominate in the document (e.g. larger score), but may not contain as much “informational content”. Also, it will give more weight to longer documents than shorter documents.

One approach is to rescale the frequency of words by how often they appear in all documents so that the scores for frequent words like “the” that are also frequent across all documents are penalized. This approach to scoring is called Term Frequency-Inverse Document Frequency, or TF-IDF for short, where:

**Term Frequency: is a scoring of the frequency of the word in the current document.**

```
TF = (Number of times term t appears in a document)/(Number of terms in the document)
```

**Inverse Document Frequency: is a scoring of how rare the word is across documents.**

```
IDF = 1+log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
```
### Cosine Similarity

Tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus

```
Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||
```
where d1,d2 are two non zero vectors.



To generate a response from our bot for input questions, the concept of document similarity will be used. We define a function response which searches the user’s utterance for one or more known keywords and returns one of several possible responses. If it doesn’t find the input matching any of the keywords, it returns a response:” I am sorry! I don’t understand you”

In [10]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Finally, we will feed the lines that we want our bot to say while starting and ending a conversation depending upon user’s input.

In [11]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
corona
ROBO: They get the name given their crown-like (“corona”) shape.
virus
ROBO: Most viruses will die, because the host is so different from humans, that it can not work.
chatbot
ROBO: I am sorry! I don't understand you
corona virus
ROBO: They get the name given their crown-like (“corona”) shape.
hello
ROBO: hi there
how are you
ROBO: I am sorry! I don't understand you
how
ROBO: I am sorry! I don't understand you
what
ROBO: I am sorry! I don't understand you
good morning
ROBO: What should one buy or do regularly So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people.
covid
ROBO: I am sorry! I don't understand you
covid 19
ROBO: I am sorry! I don't understand you
water
ROBO: I am sorry! I don't understand you
freezer
R

In [12]:
!pip install allennlp

In [18]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("config.json")

ReadError: not a gzip file

passage =""" Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[7] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[8][9] Common symptoms include fever, cough, and shortness of breath.[10] Other symptoms may include muscle pain, sputum production, diarrhea, sore throat, loss of smell, and abdominal pain.[4][11][12] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.[8][13] As of 28 March 2020, the overall rate of deaths per number of diagnosed cases is 4.7 percent; ranging from 0.2 percent to 15 percent according to age group and other health problems.[14] In comparison, the mortality rate of the 1918 flu pandemic was approximately 3% to 5%.[15]
The virus is spread mainly through close contact and via respiratory droplets produced when people cough or sneeze.[16][17] Respiratory droplets may be produced during breathing but the virus is not generally airborne.[16][18] People may also contract COVID-19 by touching a contaminated surface and then their face.[16][17] It is most contagious when people are symptomatic, although spread may be possible before symptoms appear.[17] The virus can survive on surfaces up to 72 hours.[19] Time from exposure to onset of symptoms is generally between two and fourteen days, with an average of five days.[10][20] The standard method of diagnosis is by reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[21] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.[22][23]"""

In [15]:
passage ="""How dangerous is the coronavirus?
Covid-19 is deadly, although fatality rates skyrocket for the elderly and those with compromised immune systems.

How has the coronavirus affected you?
Well if you ask me personally, the outbreak of novel Coronavirus, has affected me significantly. Both in respect of career and in respect of mental health.

Is there a treatment for the Coronavirus?
They are experimenting with various medicines and it seems some anti-malarial drugs show promise. However, they help only a little bit, as evidenced by the mortality rate in places like Italy or Spain. They are best if administered early, before the lungs are damaged.

Is there a vaccine for the coronavirus?
We don’t “find” vaccines. That’s the language yellow media uses, but it’s far from the truth.
We know the virus, and we know what it looks like. Now we need to produce a sufficient amount of attenuated virii. For that, we inject the virus into a foreign host, like a chicken embryo in an egg. Most viruses will die, because the host is so different from humans, that it can not work. Some will mutate to work, and those multiply.

How did coronaviruses get their name?
The coronavirus family was discovered first in 1960s but we don’t know where they come from. They get the name given their crown-like (“corona”) shape. Following a December 2019 outbreak in China, the World Health Organization identified a new type called 2019 novel coronavirus (2019-nCoV).

What are coronaviruses?
Coronavirus is a name for a group of viruses that target mammals, including cats, dogs, and humans. They’re well known for causing potentially fatal conditions like kidney failure, pneumonia but also mundane things like some instances of the common cold.


Does coronavirus cause death?
Statistically speaking, you have a 98% chance of survival, and 80% chance that you won’t need medical care at all even if you get infected, so stay positive, stay at home as much as possible, eat healthy, treat everyone as if they are infected, treat other as if you are infected. Many people get infected and get better without even knowing they had it.
 
What can one do to prepare for the coronavirus? What should one buy or do regularly?
So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.

What should I buy to prepare for a coronavirus epidemic if I had to stay in my home for a very long time?
So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.

What are the symptoms of Coronavirus?
The symptoms of this virus include high fever, respiratory problems, shortness of breath and coughing. In extreme cases, it can even lead to pneumonia, kidney failure and death.
To prevent infection from spreading, one should wash hands regularly, cover mouth and nose while coughing and sneezing and eat properly cook meat and eggs."""

In [16]:
result=predictor.predict(
  passage=passage,
  question="how are coronavirus?"
)
result['best_span_str']

NameError: name 'predictor' is not defined

In [17]:
result=predictor.predict(
  passage=passage,
  question="what is symptoms"
)
result['best_span_str']

NameError: name 'predictor' is not defined

In [18]:
result=predictor.predict(
  passage=passage,
  question=" death reason"
)
result['best_span_str']

NameError: name 'predictor' is not defined

In [19]:
result=predictor.predict(
  passage=passage,
  question="How dangerous is the coronavirus"
)
result['best_span_str']

NameError: name 'predictor' is not defined

### Switching to another Method to try the best process

Multidomain sentimental Data set - 
http://www.cs.jhu.edu/~mdredze/datasets/sentiment/

#  Web Scraping 
##collecting some data from twitter

In [20]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tweepy as tw
import pandas as pd
import json

# App Auth
consumer_key = 'XXX'
consumer_secret = 'XXX'
access_key = 'XXX'
access_secret = 'XXX'

# Initialize API
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Search terms
search_words = ["#coronavirus", "#COVID19", "#CoronavirusOutbreak"]
date_since = "2020-02-01"

# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since, tweet_mode='extended',
              include_rts=True).items(1000)

tweets_arr = []

# Iterate and print tweets
for tweet in tweets:
    tweets_arr.append(tweet._json)
print("Done")

with open("data.json", "w+") as output:
    output.write(json.dumps(tweets_arr))

In [ ]:
# preprocessor data sources and instances
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
tf = TfidfVectorizer()


In [ ]:
Y = []
sentences = []
sentences_processed = []
sentences_processed_tw = []

In [ ]:

# Reading data and creating sentences from data source
with open("./data/processed_stars_all") as data:
    all_data = data.read().strip().split("\n")
    for n, line in enumerate(all_data):
        line_data = line.split(" ")[0:-1]
        sentence = ""
        for words_count in line_data:
            words = words_count.split(":")[0].split("_")
            count = int(words_count.split(":")[1])
            words = list(filter("<num>".__ne__, words))
            words = list(map(lambda x: re.sub(r'\`|\'|,|\.|\"', '', x), words))

# ----------------------------------------------------------------------------

In [ ]:
df_ny_tweet_overtime = None

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in sorted(filenames)[:-2]:
        print(f"processing {filename}")
        filepath = os.path.join(dirname, filename)
        df = pd.read_csv(filepath,
                         usecols=["created_at", "text", "place_full_name"]
                        )
        df = df[df["place_full_name"].str.contains("NY", na=False)]
        if df_ny_tweet_overtime is None:
            df_ny_tweet_overtime = df
        else:
            df_ny_tweet_overtime = pd.concat([df_ny_tweet_overtime, df])

In [ ]:
df_ny_tweet_overtime.info()

In [ ]:
df_ny_tweet_overtime["created_at"] = pd.to_datetime(df_ny_tweet_overtime["created_at"])
df_ny_tweet_overtime["creation_date"] = df_ny_tweet_overtime["created_at"].dt.date

df_ny_tweet_overtime = df_ny_tweet_overtime.reset_index()
df_ny_tweet_overtime = df_ny_tweet_overtime.drop("index", axis=1)

In [ ]:
stopwords.update(["Covid19", "Covid 19", "Covid_19", "CoronaVirus", "Coronavirus", "CoronavirusOutbreak", "https"])
               
def get_word_count(data):
    return WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=50,
        max_font_size=40, 
        scale=5,
        random_state=1,
        collocations=False
    ).generate(str(data))

In [ ]:
dates, snapshots = zip(*[ (date, get_word_count('\n'.join(group["text"]))) \
                  for date, group in df_ny_tweet_overtime.groupby("creation_date") ])
frames = np.arange(len(dates))
fig = plt.figure(figsize=(15,15))
ax = fig.add_axes([0,0,1,1])
ax.axis('off')
a = snapshots[0]
im = ax.imshow(a)
ttl = ax.set_title(dates[0], fontdict={"fontsize": 100, "color": "r"})

def animate_func(i):
    im.set_array(snapshots[i])
    ttl.set_text(dates[i])
    return [im]
anim = animation.FuncAnimation(
                               fig, 
                               animate_func, 
                               frames = frames,
                               interval = 1000,
                               )


# Flash Animation of most used words regarding Coronavirus

In [ ]:
anim

In [ ]:
anim.save("anim.gif", writer="imagemagick")

# Sentiment Analysis

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
df_ny_tweet_overtime.head()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *

from nltk import tokenize

sid = SentimentIntensityAnalyzer()

df_ny_tweet_overtime['sentiment_compound_polarity']=df_ny_tweet_overtime.text.apply(lambda x:sid.polarity_scores(x)['compound'])
df_ny_tweet_overtime['sentiment_neutral']=df_ny_tweet_overtime.text.apply(lambda x:sid.polarity_scores(x)['neu'])
df_ny_tweet_overtime['sentiment_negative']=df_ny_tweet_overtime.text.apply(lambda x:sid.polarity_scores(x)['neg'])
df_ny_tweet_overtime['sentiment_pos']=df_ny_tweet_overtime.text.apply(lambda x:sid.polarity_scores(x)['pos'])
df_ny_tweet_overtime['sentiment_type']=''
df_ny_tweet_overtime.loc[df_ny_tweet_overtime.sentiment_compound_polarity>0,'sentiment_type']='POSITIVE'
df_ny_tweet_overtime.loc[df_ny_tweet_overtime.sentiment_compound_polarity==0,'sentiment_type']='NEUTRAL'
df_ny_tweet_overtime.loc[df_ny_tweet_overtime.sentiment_compound_polarity<0,'sentiment_type']='NEGATIVE'
df_ny_tweet_overtime.head()

In [ ]:
df_ny_tweet_overtime.to_csv('tweet.csv')

In [ ]:
df_ny_tweet_overtime.sentiment_type.value_counts().plot(kind='bar',title="sentiment analysis")

In [ ]:
pip install ggplot

# QnA Model using Pytorch and Bert

In [ ]:
pip install torch

In [ ]:
pip install flask

In [ ]:
pip install git+https://github.com/huggingface/pytorch-pretrained-BERT.git

My Model - https://drive.google.com/uc?id=15HOJmRizBrgoPPVDHKpvSO2tNf0k-d8f&export=download

In [ ]:
python server.py